In [7]:
!pip install SimpleITK nibabel

In [7]:
import tarfile
import tempfile
import urllib.request
from pathlib import Path
from configparser import ConfigParser
import os

def get_data_url_from_model_zoo():
    url = 'https://raw.githubusercontent.com/NifTK/NiftyNetModelZoo/5-reorganising-with-lfs/highres3dnet_brain_parcellation/main.ini'
    with urllib.request.urlopen(url) as response:
        config_string = response.read().decode()
    config = ConfigParser()
    config.read_string(config_string)
    data_url = config['data']['url']
    return data_url


def download_data(data_url):
    # tempdir = Path(tempfile.gettempdir())
    tempdir = Path(os.getcwd())
    download_dir = tempdir / 'downloaded_data'
    download_dir.mkdir(exist_ok=True)
    data_path = download_dir / Path(data_url).name
    print(data_path)
    if not data_path.is_file():
        urllib.request.urlretrieve(data_url, data_path)
    with tarfile.open(data_path, 'r') as tar:
        tar.extractall(download_dir)
    nifti_files = download_dir.glob('**/*.nii.gz')
    return list(nifti_files)[0]


def test_infer():
    image_path = download_data(get_data_url_from_model_zoo())

test_infer()

D:\workspace\dwm\jupyter_basic\ch10_monai\downloaded_data\data.tar.gz


In [8]:
import nibabel as nib
import torch
import numpy as np
import SimpleITK as sitk

In [9]:
repo = 'fepegar/highresnet'
model_name = 'highres3dnet'
print(torch.hub.help(repo, model_name))
"HighRes3DNet by Li et al. 2017 for T1-MRI brain parcellation"
"pretrained (bool): load parameters from pretrained model"
model = torch.hub.load(repo, model_name, pretrained=True)


    HighRes3DNet by Li et al. 2017 for T1-MRI brain parcellation
    pretrained (bool): load parameters from pretrained model
    


Using cache found in C:\Users\Daewoon/.cache\torch\hub\fepegar_highresnet_master
Using cache found in C:\Users\Daewoon/.cache\torch\hub\fepegar_highresnet_master


In [26]:

def resample_spacing(nifti, output_spacing, interpolation):
    output_spacing = tuple(output_spacing)
    temp_dir = Path(tempfile.gettempdir()) / '.deepgif'
    temp_dir.mkdir(exist_ok=True)
    temp_path = temp_dir / 'deepgif_resampled.nii'
    temp_path = str(temp_path)

    nifti.to_filename(temp_path)
    image = sitk.ReadImage(temp_path)

    output_spacing = np.array(output_spacing).astype(float)
    output_spacing = tuple(output_spacing)

    reference_spacing = np.array(image.GetSpacing())
    reference_size = np.array(image.GetSize())

    output_size = reference_spacing / output_spacing * reference_size
    output_size = np.round(output_size).astype(np.uint32)
    # tuple(output_size) does not work, see
    # https://github.com/Radiomics/pyradiomics/issues/204
    output_size = output_size.tolist()

    identity = sitk.Transform(3, sitk.sitkIdentity)

    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(interpolation)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())  # TODO: double-check that this is correct
    resample.SetOutputPixelType(image.GetPixelID())
    resample.SetOutputSpacing(output_spacing)
    resample.SetSize(output_size)
    resample.SetTransform(identity)
    resampled = resample.Execute(image)
    sitk.WriteImage(resampled, temp_path)
    nifti_resampled = nib.load(temp_path)
    return nifti_resampled
    
def check_header(nifti_image):
    orientation = ''.join(nib.aff2axcodes(nifti_image.affine))
    spacing = nifti_image.header.get_zooms()[:3]
    one_iso = 1, 1, 1
    # print(f'spacing and one_iso: {spacing} {one_iso}')
    is_ras = orientation == 'RAS'
    if not is_ras:
        print(f'Detected orientation: {orientation}. Reorienting to RAS...')
    is_1_iso = np.allclose(spacing, one_iso)
    if not is_1_iso:
        print(f'Detected spacing: {spacing}. Resampling to 1 mm iso...')
    needs_resampling = not is_ras or not is_1_iso
    return needs_resampling

def resample_ras_1mm_iso(nifti, interpolation=None):
    if interpolation is None:
        interpolation = sitk.sitkLinear
    nii_ras = nib.as_closest_canonical(nifti)
    spacing = nii_ras.header.get_zooms()[:3]
    one_iso = 1, 1, 1
    if np.allclose(spacing, one_iso):
        return nii_ras
    nii_resampled = resample_spacing(
        nii_ras,
        output_spacing=one_iso,
        interpolation=interpolation,
    )
    return nii_resampled


In [47]:
import numpy.ma as ma

def mean_plus(data):
    return data > data.mean()

def whiten(data, masking_function=None):
    if masking_function is None:
        masking_function = mean_plus
    mask_data = masking_function(data)
    values = data[mask_data]
    mean, std = values.mean(), values.std()
    data -= mean
    data /= std
    return data

def __compute_percentiles(img, mask, cutoff):
    """
    Creates the list of percentile values to be used as landmarks for the
    linear fitting.

    :param img: Image on which to determine the percentiles
    :param mask: Mask to use over the image to constraint to the relevant
    information
    :param cutoff: Values of the minimum and maximum percentiles to use for
    the linear fitting
    :return perc_results: list of percentiles value for the given image over
    the mask
    """
    perc = [cutoff[0],
            0.1, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.9,
            cutoff[1]]
    masked_img = ma.masked_array(img, np.logical_not(mask)).compressed()
    perc_results = np.percentile(masked_img, 100 * np.array(perc))
    return perc_results

def __standardise_cutoff(cutoff, type_hist='percentile'):
    """
    Standardises the cutoff values given in the configuration

    :param cutoff:
    :param type_hist: Type of landmark normalisation chosen (median,
    quartile, percentile)
    :return cutoff: cutoff with appropriate adapted values
    """
    cutoff = np.asarray(cutoff)
    if cutoff is None:
        return DEFAULT_CUTOFF
    if len(cutoff) > 2:
        cutoff = np.unique([np.min(cutoff), np.max(cutoff)])
    if len(cutoff) < 2:
        return DEFAULT_CUTOFF
    if cutoff[0] > cutoff[1]:
        cutoff[0], cutoff[1] = cutoff[1], cutoff[0]
    cutoff[0] = max(0., cutoff[0])
    cutoff[1] = min(1., cutoff[1])
    if type_hist == 'quartile':
        cutoff[0] = np.min([cutoff[0], 0.24])
        cutoff[1] = np.max([cutoff[1], 0.76])
    else:
        cutoff[0] = np.min([cutoff[0], 0.09])
        cutoff[1] = np.max([cutoff[1], 0.91])
    return cutoff

DEFAULT_CUTOFF = (0.01, 0.99)

def normalize(data, landmarks, cutoff=DEFAULT_CUTOFF, masking_function=None):
    mapping = landmarks

    img = data
    image_shape = img.shape
    img = img.reshape(-1).astype(np.float32)

    if masking_function is not None:
        mask = masking_function(img)
    else:
        mask = np.ones_like(img, dtype=np.bool)
    mask = mask.reshape(-1)

    range_to_use = [0, 1, 2, 4, 5, 6, 7, 8, 10, 11, 12]

    cutoff = __standardise_cutoff(cutoff)
    perc = __compute_percentiles(img, mask, cutoff)

    # Apply linear histogram standardisation
    range_mapping = mapping[range_to_use]
    range_perc = perc[range_to_use]
    diff_mapping = range_mapping[1:] - range_mapping[:-1]
    diff_perc = range_perc[1:] - range_perc[:-1]

    # handling the case where two landmarks are the same
    # for a given input image. This usually happens when
    # image background is not removed from the image.
    diff_perc[diff_perc == 0] = np.inf

    affine_map = np.zeros([2, len(range_to_use) - 1])
    # compute slopes of the linear models
    affine_map[0] = diff_mapping / diff_perc
    # compute intercepts of the linear models
    affine_map[1] = range_mapping[:-1] - affine_map[0] * range_perc[:-1]

    bin_id = np.digitize(img, range_perc[1:-1], right=False)
    lin_img = affine_map[0, bin_id]
    aff_img = affine_map[1, bin_id]
    new_img = lin_img * img + aff_img
    new_img = new_img.reshape(image_shape)

    return new_img


# From NiftyNet model zoo
LI_LANDMARKS = "4.4408920985e-16 8.06305571158 15.5085721044 18.7007018006 21.5032879029 26.1413278906 29.9862059045 33.8384058795 38.1891334787 40.7217966068 44.0109152758 58.3906435207 100.0"
LI_LANDMARKS = np.array([float(n) for n in LI_LANDMARKS.split()])

def standardize(data, landmarks=LI_LANDMARKS, masking_function=None):
    return normalize(data, landmarks, masking_function=masking_function)

def pad(data, padding):
    # Should I use this value for padding?
    value = data[0, 0, 0]
    return np.pad(data, padding, mode='constant', constant_values=value)

def preprocess(data, padding, hist_masking_function=None):
    # data = pad(data, padding)
    data = standardize(data, masking_function=hist_masking_function)
    data = whiten(data)
    data = data.astype(np.float32)
    data = pad(data, padding)  # should I pad at the beginning instead?
    return data

In [39]:
input_path1 = "_data/GAAIN/AD01_MR.nii"
# nii1 = nib.load(str(input_path1))
input_path2 = "_data/OASIS/OAS1_MR.nii.gz"
input_path3 = "_data/SNU/P01_MR.nii"
# nii2 = nib.load(str(input_path2))
# print(nii1)
# print(nii2)

In [40]:
input_path = input_path2
nii = nib.load(str(input_path))
needs_resampling = check_header(nii)
print(needs_resampling)

Detected orientation: ASL. Reorienting to RAS...
True


In [41]:
if needs_resampling:
    nii = resample_ras_1mm_iso(nii)
data = nii.get_fdata()

In [48]:
use_niftynet_hist_std = False
# Preprocessing
hist_masking_function = mean_plus if use_niftynet_hist_std else None
volume_padding = int(10)
preprocessed = preprocess(
    data,
    volume_padding,
    hist_masking_function=hist_masking_function,
)

d:\workspace\dwm\jupyter_basic\venv\lib\site-packages\ipykernel_launcher.py:76: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
